In [12]:
# !pip install llama-index
# !pip install openai
# !pip install pypdf

In [22]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
import os

# LOAD THE DOCUMENTS
root_dir_local = '/Users/brauliopf/'
root_dir_cloud = '/Users/brauliopf/brauliopf@gmail.com - Google Drive/My Drive/'
root_dir_cloudUM = '/Users/brauliopf/braulio@umich.edu - Google Drive/My Drive/'
# os.listdir(root_dir_local) # list all files in the directory

## Loading
documents = SimpleDirectoryReader(root_dir_local + 'Documents/Dev/old/100-MLBook/').load_data()

# BUILD THE INDEX
try:
    # Loading Index from local "./storage". Run this cell if you want to load an index to resume
    from llama_index import StorageContext, load_index_from_storage

    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)
except:
    # This will create (or replace) a "./storage" folder and save the index there
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist() # Save your index to a index.json file

been here


In [21]:
# QUERY THE INDEX
query_engine = index.as_query_engine()
response = query_engine.query("How can I address overfitting in my LLM?")
print(response)


One way to address overfitting in a LLM is to use regularization. This involves modifying the objective function by adding a penalizing term whose value is higher when the model is more complex. Additionally, one can try a simpler model, reduce the dimensionality of examples in the dataset, add more training data, or regularize the model.
